#Load Data

In [2]:
conda install numpy
conda install -c intel mkl

SyntaxError: invalid syntax (<ipython-input-2-d6c43e242201>, line 1)

In [ ]:
# Load Tweets
import pandas as pd
pd.set_option('max_colwidth', 20)

trumpTweets = pd.read_json('trump.json', lines=True)

pd.set_option('max_colwidth', 400)

In [1]:
# Filter the dataframes to show id, content and date
trumpTweets = trumpTweets.filter(['id','content','date'], axis=1)

NameError: name 'trumpTweets' is not defined

In [ ]:
#Inspect tweets
display(trumpTweets.head(10))
display(trumpTweets.info())

#Extract Emoji's

In [ ]:
!pip3 install -U emoji

In [ ]:
# Import the packages we need
import emoji
import regex

# Define a function to get emojis
'Function that extracts the emojis from a text and returns them in a list'
def get_emojis(text):
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI["en"] for char in word):
            emoji_list.append(word)

    return emoji_list

In [ ]:
# Let's do that for all of our tweets and store the emojis in a new column called "emojis"

trumpTweets['emojis'] = trumpTweets['content'].apply(get_emojis)

# Check if it worked
display(trumpTweets.iloc[340:352,:]) # logical indexing: from tweets get rows 340,352

#Remove Undesirable Characters

In [ ]:
# import regular expressions
import re

# Set-up patterns to be removed fro the tweets
pat1 = r"http\S+"
pat2 = r"#"
pat3 = r"@"
pat4 = r"FAV"
pat5 = r"RE"
pat6 = r"pic.\S+"
pat7 = r"\n"
pat8 = '\r\n'
pat9 = r'|'.join((r'&amp;',r'&copy;',r'&reg;',r'&quot;',r'&gt;',r'&lt;',r'&nbsp;',r'&apos;',r'&cent;',r'&euro;',r'&pound;'))
# Combine all patterns
combined_pat = r'|'.join((pat1, pat2, pat3, pat4, pat5, pat6, pat7, pat8, pat9))

# replace the patterns with an empty string
trumpTweets['stripped'] =  [re.sub(combined_pat, '', w) for w in trumpTweets.content]
# might have double spaces now (because of empty string replacements above) - remove double empty spaces
trumpTweets['stripped']  = [re.sub(r"  ", ' ', w) for w in trumpTweets.loc[:,'stripped']]

#Apply Sentiment Scores to Each Tweet

In [ ]:
!pip install vaderSentiment

In [ ]:
# Import the sentiment module (if you haven't already done so)
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Import numpy (if you have not already done so)
import numpy as np

# Instantiate the sentiment analyzer (if you haven't already done so)
analyser = SentimentIntensityAnalyzer()

# Now get the compound sentiment score for each tweet
trumpTweets['C_Score'] = np.nan
for index, row in trumpTweets.iterrows(): 
    trumpTweets.loc[index, 'C_Score'] = analyser.polarity_scores(row['stripped'])['compound']
# Let's take a look!
pd.set_option('display.max_colwidth', None)

In [ ]:
trumpTweets[['stripped','C_Score']][540:552]

#Cumulative Statistics

In [ ]:
# import necessary modules (if not already imported)
import pandas as pd
import numpy as np

print(f"Count positive tweets: {sum(trumpTweets['C_Score'] > 0.05)}")
print(f"Count netural tweets: {trumpTweets['C_Score'].between(-0.05, 0.05).sum()}")
print(f"Count negative tweets: {sum(trumpTweets['C_Score'] < -0.05)}")
print(f"Total number of tweets: {trumpTweets['C_Score'].count()}")
      
display(trumpTweets.C_Score.describe())

#Sentiment Distribution

In [ ]:
# import necessary modules (if not already imported)
import matplotlib.pyplot as plt
import seaborn as sns

# settings for seaborn plotting style
sns.set(color_codes=True)

# settings for seaborn plot sizes
sns.set(rc={'figure.figsize':(5,5)})

# Create Histogram
ax = sns.histplot(trumpTweets['C_Score'],
                  bins=10,
                  kde=False,
                  color='skyblue')
ax.set(xlabel='Sentiment Distribution', ylabel='Frequency')

In [ ]:
# create new column
trumpTweets['Sentiment'] = np.nan

# Loop through rows of dataframe and determine strings for new column "Sentiment"
for index, row in trumpTweets.iterrows(): 
    if trumpTweets.loc[index, 'C_Score'] > 0.05 : 
            trumpTweets.loc[index, 'Sentiment'] = "Positive"       
    elif trumpTweets.loc[index, 'C_Score'] < -0.05 :
            trumpTweets.loc[index, 'Sentiment'] = "Negative"   
    else : 
        trumpTweets.loc[index, 'Sentiment'] = "Neutral"

trumpTweets['Sentiment'] = trumpTweets['Sentiment'].astype("category")

In [ ]:
# See whether it worked 
trumpTweets[['stripped','C_Score', 'Sentiment']].tail(25)


#Sentiment Distribution over Time

In [ ]:
import math
trumpTweets['day'] = [one.date() for one in trumpTweets['date']]
trumpTweets = trumpTweets.sort_values(by=['day'])

sentiments = ["Positive", "Neutral", "Negative"]
positiveProps = (trumpTweets[trumpTweets.Sentiment == 'Positive'].groupby(['day']).count()[['Sentiment']]/ trumpTweets.groupby(['day']).count()[['Sentiment']])*100
neutralProps = (trumpTweets[trumpTweets.Sentiment == 'Neutral'].groupby(['day']).count()[['Sentiment']]/ trumpTweets.groupby(['day']).count()[['Sentiment']])*100
negativeProps = (trumpTweets[trumpTweets.Sentiment == 'Negative'].groupby(['day']).count()[['Sentiment']]/ trumpTweets.groupby(['day']).count()[['Sentiment']])*100
 
positiveProps = positiveProps['Sentiment'].tolist()
neutralProps = neutralProps['Sentiment'].tolist()
negativeProps = negativeProps['Sentiment'].tolist()
plt.figure(figsize=[16, 5])
barWidth = 0.5
labels = trumpTweets.day.unique()
r = np.arange(len(labels))
positiveProps = [0 if math.isnan(x) else x for x in positiveProps]
neutralProps = [0 if math.isnan(x) else x for x in neutralProps]
negativeProps = [0 if math.isnan(x) else x for x in negativeProps]

plt.bar(r,positiveProps, color='lightgreen', edgecolor='white', width=barWidth)
plt.bar(r, neutralProps, bottom=positiveProps, color='skyblue', edgecolor='white', width=barWidth)
plt.bar(r, negativeProps, bottom=[i+j for i,j in zip(positiveProps, neutralProps)], color='orange', edgecolor='white', width=barWidth)
 
plt.xticks(r, labels, rotation = 45, fontsize=12)
plt.yticks(fontsize=16)
plt.suptitle('Sentiment Distribution over Time')
plt.xlabel("Date", fontsize=18)
plt.ylabel("Share", fontsize=20)
plt.legend(sentiments)
plt.show()
# Sort by Index again
trumpTweets.sort_index(inplace=True)

#Tweet Preprocessor

In [ ]:
# 1. You need to install it first:
!pip3 install tweet-preprocessor

In [ ]:
# 2. Preprocess the tweets

# Import the preprocessor
import preprocessor as prepro

# Set options to remove URL, Reserved word
prepro.set_options(prepro.OPT.URL, prepro.OPT.RESERVED)

# Let's do it for all tweets
trumpTweets['text']  = trumpTweets['content'].apply(prepro.clean)

# Check our work
trumpTweets['text'].tail(5)

In [ ]:
# 3. Fix some things the preprocessor missed

# Replace html &amp; with and
trumpTweets['text'] =  [re.sub(r'&amp;', 'and', w) for w in trumpTweets.text]

# Remove other html entities 
htmlents = r'|'.join((r'&copy;',r'&reg;',r'&quot;',r'&gt;',r'&lt;',r'&nbsp;',r'&apos;',r'&cent;',r'&euro;',r'&pound;')) 
trumpTweets['text'] =  [re.sub(htmlents, '', w) for w in trumpTweets.text]

# Notice that Hashtags and Mentions are still in the tweets. Sometimes, we may want to treat them as nouns (e.g., for a brand or person)
# To do so, we need to remove the special characters # and @ using regular expressions

# Set-up patterns to be removed fro the tweets
htag = r'|'.join((r"#", r"@"))

# Now, replace the patterns with an empty string
trumpTweets['text'] =  [re.sub(htag, '', w) for w in trumpTweets.text]

# We might have double spaces now (because of empty string replacements above) - remove double empty spaces
#tweets['stripped']  = [re.sub(r"  ", ' ', w) for w in tweets.loc[:,'stripped']]
trumpTweets['text'] = trumpTweets.text.replace({' +':' '},regex=True)

# Check our Work
trumpTweets['text'].tail(5)

#EDA Sentiment Analysis

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 

# 1. Instantiate sentiment analyzer
sid_obj = SentimentIntensityAnalyzer()
key_list = "pos neu neg compound overall_sentiment".split(" ")
for i in key_list:
  trumpTweets[i] = ""

# 2. Analyze the tweets text row by row in our dataframe and fill in it's sentiment, polarity, and overall sentiment
for index, row in trumpTweets.iterrows():
    row_sentiment_dict = sid_obj.polarity_scores(row['text'])
    
    if row_sentiment_dict['compound'] >= 0.05 : 
        row_sentiment_dict['overall_sentiment']  = "Positive"

    elif row_sentiment_dict['compound'] <= - 0.05 : 
        row_sentiment_dict['overall_sentiment'] =  "Negative"

    else : 
        row_sentiment_dict['overall_sentiment'] =  "Neutral"
    for i in key_list:
        trumpTweets.loc[index, i] = row_sentiment_dict[i]


In [ ]:
# 3. Let's check the overall sentiment distribution
trumpTweets['overall_sentiment'].value_counts()

In [ ]:
# Create List of Competitors
Competitors = trumpTweets['query_text'].unique()

# Create Stacked Bar Chart
import matplotlib.pyplot as plt
r = range(len(Competitors))
sentiments = ["positive", "neutral", "negative"]

plt.figure(figsize=[10, 5])

positiveProps = (trumpTweets[trumpTweets.overall_sentiment == 'Positive'].groupby(['query_text']).count()[['overall_sentiment']]/ trumpTweets.groupby(['query_text']).count()[['overall_sentiment']])*100
neutralProps = (trumpTweets[trumpTweets.overall_sentiment == 'Neutral'].groupby(['query_text']).count()[['overall_sentiment']]/ trumpTweets.groupby(['query_text']).count()[['overall_sentiment']])*100
negativeProps = (trumpTweets[trumpTweets.overall_sentiment == 'Negative'].groupby(['query_text']).count()[['overall_sentiment']]/ trumpTweets.groupby(['query_text']).count()[['overall_sentiment']])*100
 
positiveProps = positiveProps['overall_sentiment'].tolist()
neutralProps = neutralProps['overall_sentiment'].tolist()
negativeProps = negativeProps['overall_sentiment'].tolist()

barWidth = 0.5
labels = trumpTweets.groupby(['query_text']).query_text.unique().keys()
plt.bar(r,positiveProps, color='#b5ffb9', edgecolor='white', width=barWidth)
plt.bar(r, neutralProps, bottom=positiveProps, color='#FFFF99', edgecolor='white', width=barWidth)
plt.bar(r, negativeProps, bottom=[i+j for i,j in zip(positiveProps, neutralProps)], color='#F08080', edgecolor='white', width=barWidth)
 
plt.xticks(r, labels, rotation = 45, fontsize=16)
plt.yticks(fontsize=16)
plt.suptitle('Distribution of Sentiment By Competitor')
plt.xlabel("Competitor", fontsize=18)
plt.ylabel("Share of Tweets", fontsize=20)
plt.legend(sentiments)
plt.show()

# Word Clouds

In [ ]:
'Function to generate Word Cloud object'
def plotWorldCloud(keyword):
    df = trumpTweets[trumpTweets['query_text'] == keyword]
    all_words = ' '.join([text for text in df[df['overall_sentiment'] == 'Positive']['text']])
    if len(all_words) > 5:
        wordcloud = WordCloud(collocations=True, width=800, height=500, random_state=21, max_font_size=110).generate(all_words)
        return wordcloud
    else:
        return False

# List of Competitors   
Competitors = trumpTweets['query_text'].unique()

# Build Grid
gridSize = math.ceil(math.sqrt(len(Competitors)))

fig = plt.figure(figsize=(30, 20))
i = 0
d={}
for keyword in Competitors:
    d["ax{0}".format(i)]=fig.add_subplot(gridSize,gridSize,i+1)
    wordcloud = plotWorldCloud(keyword)
    if wordcloud:
        d["ax{0}".format(i)].title.set_text(keyword)
        d["ax{0}".format(i)].title.set_fontsize(30)
        d["ax{0}".format(i)].imshow(wordcloud)
        d["ax{0}".format(i)].axis('off')
        i = i + 1
    